In [6]:
import pandas as pd
import mysql.connector

from IPython.display import display, Markdown, HTML

db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="secret",
    port="3306",
    database="serlo"
)

def cached(func):
    cache = dict()
    
    def return_func(arg):
        if (arg in cache):
            return cache[arg]
        else:
            result = func(arg)
            cache[arg] = result
            return result
    
    return return_func

def query(sql):
    c = db.cursor()
    c.execute(sql)
    
    return c.fetchall()

def querySingleton(sql):
    return [ x[0] for x in query(sql) ]

def read_all_current_contents():
    return pd.read_sql("""
        select entity.id, entity_revision_field.value as content from entity
        join entity_revision on entity_revision.id = entity.current_revision_id
        join entity_revision_field on entity_revision_field.entity_revision_id = entity_revision.id
        where entity_revision_field.field  = "content"
    """, db)

df = read_all_current_contents()
df[df["content"] == ""]

,id,content
6539,14573,


In [9]:
def read_all_current_contents_without_deleted():
    return pd.read_sql("""
        select entity.id, entity_revision_field.value as content from entity
        join uuid on uuid.id = entity.id
        join entity_revision on entity_revision.id = entity.current_revision_id
        join entity_revision_field on entity_revision_field.entity_revision_id = entity_revision.id
        where entity_revision_field.field  = "content"
        and uuid.trashed = 0
    """, db)

df = read_all_current_contents_without_deleted()
df[df["content"] == ""]

,id,content
6457,14573,
